# Unified Bombcell Runner (Grant)

Single entrypoint replacing batch, single-probe, and NP2.0 rerun notebooks.

In [14]:
CONFIG_FILE = r'C:\Users\user\Documents\github\bombcell\py_bombcell\grant\configs\reach15_20260201_session007_config_155min_region_specific.json'

RUN_MODE = 'batch'  # batch | single_probe | np20_rerun
TARGET_PROBE = 'F'  # only used for single_probe
OVERWRITE = True


In [15]:
from pathlib import Path
import subprocess

runner = Path('run_bombcell_unified.py')
cmd = ['python', str(runner), '--config', CONFIG_FILE, '--mode', RUN_MODE]
if RUN_MODE == 'single_probe':
    cmd += ['--target-probe', TARGET_PROBE]
if OVERWRITE:
    cmd.append('--overwrite')

print('Running:', ' '.join(cmd))
result = subprocess.run(cmd, capture_output=True, text=True)

if result.stdout:
    print(result.stdout)
if result.stderr:
    print(result.stderr)
    
if result.returncode != 0:
    print(f'\n=== Error: Command failed with exit code {result.returncode} ===')
    raise subprocess.CalledProcessError(result.returncode, cmd, result.stdout, result.stderr)

print('\n=== Completed successfully ===')

Running: python run_bombcell_unified.py --config C:\Users\user\Documents\github\bombcell\py_bombcell\grant\configs\reach15_20260201_session007_config_155min_region_specific.json --mode batch --overwrite
✅ ipywidgets available - interactive GUI ready

=== Probe A (NP2.0 Simplex Lobule & Interposed Nucleus (SIM/IP)) ===
ks_dir: H:\Grant\Neuropixels\Kilosort_Recordings\Reach15_20260201_session007_NP_Recording_Number02_2026-02-01_18-25-00\bombcell\bombcell_DEFAULT\kilosort4_A
raw_file: H:\Grant\Neuropixels\Kilosort_Recordings\Reach15_20260201_session007_NP_Recording_Number02_2026-02-01_18-25-00\Record Node 103\experiment1\recording1\continuous\Neuropix-PXI-100.ProbeA\continuous.dat
meta_file: H:\Grant\Neuropixels\Kilosort_Recordings\Reach15_20260201_session007_NP_Recording_Number02_2026-02-01_18-25-00\Record Node 103\experiment1\recording1\structure.oebin
Applying overrides for probe A: {'maxNPeaks': 4, 'maxNTroughs': 3, 'minWvDuration': 120, 'maxWvDuration': 1400, 'maxWvBaselineFraction':